In [ ]:

import dash
from dash import dcc, html, Input, Output, State
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import base64
import io
import urllib.parse
import time

# 初始化 Dash 应用
app = dash.Dash(__name__)

# 默认平滑参数：窗口比例与多项式阶数
DEFAULT_WINDOW_FRACTION = 0.03
DEFAULT_POLYORDER = 1

# 设置图表的电压范围（可根据需要调整）
X_AXIS_RANGE = (0, 1.5)


def select_soc_or_dod(df):
    """
    根据数据中是否存在有效的 SoC[%] 列，选择 SoC 或 DoD 列
    如果 SoC 全部为空，则返回 'DoD[%]'，否则返回 'SoC[%]'
    """
    if df['SoC[%]'].isna().all():
        return 'DoD[%]'
    else:
        return 'SoC[%]'


def process_files(files, window_fraction=DEFAULT_WINDOW_FRACTION, polyorder=DEFAULT_POLYORDER):
    """
    处理上传的文件列表，计算并绘制：
    - 充电与放电模式下的 dQ/dV 曲线（平滑后）
    - 充电与放电模式下的 SoC/DoD vs Voltage 曲线
    - 综合所有文件与模式的 dQ/dV 平滑曲线
    - 综合所有文件与模式的 SoC/DoD vs Voltage 曲线

    返回：图表列表、文件数量、处理耗时、数据点数、窗口长度
    """
    start_time = time.time()

    # 使用经典风格绘图
    plt.style.use('classic')
    figs = []  # 存储所有生成的 Figure 对象

    # 针对"charge"和"discharge"两种模式分别绘图
    for mode, cond in [('charge', [1, 10]), ('discharge', [2, 20])]:
        # ========== 绘制 dQ/dV 平滑曲线 ==========
        fig, ax = plt.subplots(figsize=(10, 10))
        lines = []

        for content, filename in files:
            # 解析上传的 Base64 数据并加载为 DataFrame
            _, content_string = content.split(',')
            decoded = base64.b64decode(content_string)
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))

            # 根据 Condition 列筛选当前模式的数据
            df_part = df[df['Condition'].isin(cond)].copy().reset_index(drop=True)
            if df_part.empty:
                continue

            # 计算 dQ/dV：电容量对电压的差分
            voltage = df_part['Voltage[V]']
            capacity = df_part['Capacity[mAh]']
            dQdV_raw = capacity.diff() / voltage.diff()

            # 清除无穷大，并插值补充缺失值
            dQdV_clean = dQdV_raw.replace([np.inf, -np.inf], np.nan).interpolate(limit_direction='both')

            # 初始化平滑结果数组
            smoothed = np.full_like(voltage, np.nan)
            if len(dQdV_clean) >= polyorder + 3:
                # 根据比例计算窗口长度，并取最近的奇数
                window_len = max(5, int(len(dQdV_clean) * window_fraction) // 2 * 2 + 1)
                window_len += (window_len % 2 == 0)
                try:
                    smoothed = savgol_filter(dQdV_clean, window_length=window_len, polyorder=polyorder)
                except Exception:
                    # 滤波失败时保留 NaN
                    pass

            # 绘制平滑后的曲线
            line, = ax.plot(voltage, smoothed, label=f'{filename} ({mode} smoothed)')
            lines.append(line)

        # 设置图表属性并保存
        ax.set_title(f'{mode.capitalize()} dQ/dV Curves')
        ax.set_xlabel('Voltage [V]')
        ax.set_ylabel('dQ/dV')
        ax.set_xlim(X_AXIS_RANGE)
        ax.legend(handles=lines, loc='upper right', fontsize='small')
        figs.append(fig)

        # ========== 绘制 SoC/DoD vs Voltage 曲线 ==========
        fig2, ax2 = plt.subplots(figsize=(10, 10))
        for content, filename in files:
            _, content_string = content.split(',')
            df = pd.read_csv(io.StringIO(base64.b64decode(content_string).decode('utf-8')))
            df_part = df[df['Condition'].isin(cond)].copy().reset_index(drop=True)
            if df_part.empty:
                continue

            # 根据数据选择 SoC 或 DoD 列
            col = select_soc_or_dod(df_part)
            if col not in df_part.columns or df_part[col].isna().all():
                continue

            # 绘制 SoC/DoD vs 电压
            ax2.plot(df_part[col], df_part['Voltage[V]'], label=f'{filename} ({mode})')

        ax2.set_title(f'{mode.capitalize()} SoC vs Voltage')
        ax2.set_xlabel(col)
        ax2.set_ylabel('Voltage [V]')
        ax2.legend(loc='upper right', fontsize='small')
        figs.append(fig2)

    # ========== 综合 dQ/dV 曲线 ==========
    fig3, ax3 = plt.subplots(figsize=(10, 10))
    for content, filename in files:
        _, content_string = content.split(',')
        df = pd.read_csv(io.StringIO(base64.b64decode(content_string).decode('utf-8')))
        for mode, cond in [('charge', [1, 10]), ('discharge', [2, 20])]:
            df_part = df[df['Condition'].isin(cond)].copy().reset_index(drop=True)
            if df_part.empty:
                continue

            voltage = df_part['Voltage[V]']
            capacity = df_part['Capacity[mAh]']
            dQdV_clean = (capacity.diff() / voltage.diff()).replace([np.inf, -np.inf], np.nan).interpolate(limit_direction='both')
            smoothed = savgol_filter(dQdV_clean, window_length=5, polyorder=DEFAULT_POLYORDER) if len(dQdV_clean) >= 4 else np.full_like(voltage, np.nan)
            ax3.plot(voltage, smoothed, label=f'{filename} ({mode} smoothed)')

    ax3.set_title('Combined Charge and Discharge dQ/dV Curves')
    ax3.set_xlabel('Voltage [V]')
    ax3.set_ylabel('dQ/dV')
    ax3.set_xlim(X_AXIS_RANGE)
    ax3.legend(loc='upper right', fontsize='small')
    figs.append(fig3)

    # ========== 综合 SoC/DoD vs Voltage 曲线 ==========
    fig4, ax4 = plt.subplots(figsize=(10, 10))
    for content, filename in files:
        _, content_string = content.split(',')
        df = pd.read_csv(io.StringIO(base64.b64decode(content_string).decode('utf-8')))
        for mode, cond in [('charge', [1, 10]), ('discharge', [2, 20])]:
            df_part = df[df['Condition'].isin(cond)].copy().reset_index(drop=True)
            if df_part.empty:
                continue
            col = select_soc_or_dod(df_part)
            if col not in df_part.columns or df_part[col].isna().all():
                continue
            ax4.plot(df_part[col], df_part['Voltage[V]'], label=f'{filename} ({mode})')

    ax4.set_title('Combined Charge and Discharge SoC vs Voltage')
    ax4.set_xlabel(col)
    ax4.set_ylabel('Voltage [V]')
    ax4.legend(loc='upper right', fontsize='small')
    figs.append(fig4)

    # 计算并返回总耗时、文件数等信息
    processing_time = time.time() - start_time
    return figs, len(files), processing_time, len(dQdV_clean), window_len


def plot_to_uri(fig):
    """
    将 Matplotlib Figure 转为 Base64 URI，用于在 Dash 前端展示
    """
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    string = base64.b64encode(buf.read()).decode('utf-8')
    # URL 编码后拼接成 img src
    return 'data:image/png;base64,' + urllib.parse.quote(string)

# 定义应用布局
app.layout = html.Div([
    html.H2("Battery dQ/dV 曲线 +SOC电压曲线"),
    dcc.Upload(
        id='upload-data',
        children=html.Div(['拖放或点击上传 CSV 文件']),
        style={
            'width': '98%', 'height': '60px', 'lineHeight': '60px',
            'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '5px',
            'textAlign': 'center', 'margin': '10px'
        },
        multiple=True  # 支持多文件上传
    ),
    # 信息面板 & 图像展示区域
    html.Div(id='info-panel'),
    html.Div(id='output-plots')
])

# 回调：上传文件后触发，更新信息面板和图像
@app.callback(
    [Output('info-panel', 'children'), Output('output-plots', 'children')],
    Input('upload-data', 'contents'),
    State('upload-data', 'filename')
)
def update_plots(list_of_contents, list_of_names):
    if not list_of_contents:
        return [], []

    files = list(zip(list_of_contents, list_of_names))
    # 处理文件并获取图表及统计信息
    figs, num_files, processing_time, num_points, window_len = process_files(files)
    img_uris = [plot_to_uri(fig) for fig in figs]

    # 构建信息面板
    info_panel = html.Div([
        html.P("平滑方式：Savitzky-Golay滤波"),
        html.P("polyorder指数：1"),
        html.P(f"点数：{num_points}"),
        html.P(f"平滑窗口：{window_len}"),
        html.P(f"处理csv文件数：{num_files}"),
        html.P(f"处理耗时：{processing_time:.2f}秒")
    ], style={'marginBottom': '20px'})

    # 将每个图转为 <img> 标签展示
    plots = [html.Img(src=uri, style={'width': '50%', 'margin-bottom': '100px'}) for uri in img_uris]

    return info_panel, plots

# 启动服务器
if __name__ == '__main__':
    app.run(debug=True, port=8050)


